### Consistency Check Alpaca




In [1]:
# Import libraries and dependencies
!pip install alpaca-trade-api
import numpy as np
import pandas as pd
#import os
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from joblib import dump, load
%matplotlib inline

  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninstalled urllib3-1.25.9
ERROR: holoviz 0.11.3 has requirement matplotlib==3.0.3, but you'll have matplotlib 3.1.3 which is incompatible.
ERROR: botocore 1.17.7 has requirement docutils<0.16,>=0.10, but you'll have docutils 0.16 which is incompatible.


In [2]:
# Load .nv enviroment variables
#from dotenv import load_dotenv
#load_dotenv()

# Set Alpaca API key and secret
#alpaca_api_key = os.getenv("ALPACA_API_KEY")
#alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# REPLACE THESE WITH YOUR API KEY
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

NameError: name 'alpaca_api_key' is not defined

### Check Available Tickers from Alpaca Markets API

In [ ]:
# Get all Assets
assets = api.list_assets()

# Keep only tradeable assets
tradeable = [asset for asset in assets if asset.tradable ]


In [ ]:
# Create a new empty DataFrame
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])

# Display the first 10 asset tickers
display(asset_info_df.head(10))

### Get 1 Year's Worth of Data via API Call and Read in as DataFrame 

In [5]:
# Set the list of tickers
tickers = ["AAPL", "MSFT","FB","NFLX","AMZN"]

# Pulling S&P Tickers
#table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#tickers=[table[0]['Symbol'].values][0]
#tickers = tickers[:100]

# Set timeframe to '1D' 1Min
timeframe = '1D'

# Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp("2017-5-04 09:30", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2019-5-11 16:01", tz="America/New_York").isoformat()

# Get 1 year's worth of historical data for JNJ and MU.
df = api.get_barset(
    tickers,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

df.head()
#df.tail(10)
#df.describe()

NameError: name 'api' is not defined

In [ ]:
df.isna().mean()

AAPL  open      0.0
      high      0.0
      low       0.0
      close     0.0
      volume    0.0
AMZN  open      0.0
      high      0.0
      low       0.0
      close     0.0
      volume    0.0
FB    open      0.0
      high      0.0
      low       0.0
      close     0.0
      volume    0.0
MSFT  open      0.0
      high      0.0
      low       0.0
      close     0.0
      volume    0.0
NFLX  open      0.0
      high      0.0
      low       0.0
      close     0.0
      volume    0.0
dtype: float64

### Drop Extraneous Columns

In [ ]:
# Use the `drop` function with the `level` parameter to drop extra columns in the multi-index DataFrame
df.drop(columns=['open', 'high', 'low', 'volume'], level=1, inplace=True)
df.head()

,AAPL,AMZN,FB,MSFT,NFLX
,close,close,close,close,close
2017-05-04 00:00:00-04:00,146.54,937.53,150.84,68.81,157.26
2017-05-05 00:00:00-04:00,148.96,934.15,150.20,68.99,156.59
2017-05-08 00:00:00-04:00,153.01,949.02,151.05,68.94,156.38
2017-05-09 00:00:00-04:00,153.98,952.80,150.49,69.03,157.46
2017-05-10 00:00:00-04:00,153.26,948.95,150.29,69.31,160.28


In [ ]:
returns_pct_change1 = df.pct_change(1) 
returns_pct_change2 = df.pct_change(2) 
returns_pct_change3 = df.pct_change(3) 
returns_pct_change4 = df.pct_change(4) 
returns_pct_change5 = df.pct_change(5)
returns_pct_change10 = df.pct_change(10) 
returns_pct_change30 = df.pct_change(30) 
returns_pct_change60 = df.pct_change(60) 

In [ ]:
F_prices1m = returns_pct_change1.shift(-1)
F_prices1m = pd.DataFrame(F_prices1m.unstack())
F_prices1m.rename(columns={0: 'F_1m_returns'}, inplace = True)
F_prices1m.reset_index(inplace = True)
F_prices1m.drop(columns='level_1',inplace = True)
F_prices1m.rename(columns = {'level_2' : 'level_1'}, inplace = True)
#F_prices1m.head()

prices1m = pd.DataFrame(returns_pct_change1.unstack())
prices1m.rename(columns={0: '1m_returns'}, inplace = True)
prices1m.reset_index(inplace = True)
prices1m.drop(columns='level_1',inplace = True)
prices1m.rename(columns = {'level_2' : 'level_1'},inplace = True)
prices1m.head()

prices2m = pd.DataFrame(returns_pct_change2.unstack())
prices2m.rename(columns={0: '2m_returns'}, inplace = True)
prices2m.reset_index(inplace = True)
prices2m.drop(columns='level_1',inplace = True)
prices2m.rename(columns = {'level_2' : 'level_1'},inplace = True)
#prices2m.head()

prices3m = pd.DataFrame(returns_pct_change3.unstack())
prices3m.rename(columns={0: '3m_returns'}, inplace = True)
prices3m.reset_index(inplace = True)
prices3m.drop(columns='level_1',inplace = True)
prices3m.rename(columns = {'level_2' : 'level_1'},inplace = True)
#prices3m.head()

prices4m = pd.DataFrame(returns_pct_change4.unstack())
prices4m.rename(columns={0: '4m_returns'}, inplace = True)
prices4m.reset_index(inplace = True)
prices4m.drop(columns='level_1',inplace = True)
prices4m.rename(columns = {'level_2' : 'level_1'},inplace = True)
#prices4m.head()

prices5m = pd.DataFrame(returns_pct_change5.unstack())
prices5m.rename(columns={0: '5m_returns'}, inplace = True)
prices5m.reset_index(inplace = True)
prices5m.drop(columns='level_1',inplace = True)
prices5m.rename(columns = {'level_2' : 'level_1'},inplace = True)
#prices5m.head(10)

prices10m = pd.DataFrame(returns_pct_change10.unstack())
prices10m.rename(columns={0: '10m_returns'}, inplace = True)
prices10m.reset_index(inplace = True)
prices10m.drop(columns='level_1',inplace = True)
prices10m.rename(columns = {'level_2' : 'level_1'},inplace = True)
#prices10m.head(10)

prices30m = pd.DataFrame(returns_pct_change30.unstack())
prices30m.rename(columns={0: '30m_returns'}, inplace = True)
prices30m.reset_index(inplace = True)
prices30m.drop(columns='level_1',inplace = True)
prices30m.rename(columns = {'level_2' : 'level_1'},inplace = True)
#prices30m.head(10)

prices60m = pd.DataFrame(returns_pct_change60.unstack())
prices60m.rename(columns={0: '60m_returns'}, inplace = True)
prices60m.reset_index(inplace = True)
prices60m.drop(columns='level_1',inplace = True)
prices60m.rename(columns = {'level_2' : 'level_1'},inplace = True)
#prices60m.head(10)
prices1m.head()

,level_0,level_1,1m_returns
0,AAPL,2017-05-04 00:00:00-04:00,NaN
1,AAPL,2017-05-05 00:00:00-04:00,0.016514
2,AAPL,2017-05-08 00:00:00-04:00,0.027189
3,AAPL,2017-05-09 00:00:00-04:00,0.006339
4,AAPL,2017-05-10 00:00:00-04:00,-0.004676


In [ ]:
prices1m.tail()
prices1m.shape

(25, 3)

In [ ]:
prices1m= prices1m.iloc[:10,:]
prices2m= prices2m.iloc[:10,:]
prices3m= prices3m.iloc[:10,:]
prices4m= prices4m.iloc[:10,:]
prices5m= prices5m.iloc[:10,:]
prices10m= prices10m.iloc[:10,:]
prices30m= prices30m.iloc[:10,:]
prices60m= prices60m.iloc[:10,:]

In [ ]:
data12 = pd.merge(prices1m,prices2m,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))
data123 = pd.merge(data12,prices3m,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))
data1234 = pd.merge(data123,prices4m,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))
data12345 = pd.merge(data1234,prices5m,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))
data1234510 = pd.merge(data12345,prices10m,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))
data123451030 = pd.merge(data1234510,prices30m,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))
data12345103060 = pd.merge(data123451030,prices60m,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))
dataFinal = pd.merge(data12345103060,F_prices1m,left_on=['level_0', 'level_1'],right_on=['level_0', 'level_1'], how='left', suffixes=('_original', 'right'))
dataFinal.head(100)


,level_0,level_1,1m_returns,2m_returns,3m_returns,4m_returns,5m_returns,10m_returns,30m_returns,60m_returns,F_1m_returns
0,AAPL,2017-05-04 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.016514
1,AAPL,2017-05-05 00:00:00-04:00,0.016514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027189
2,AAPL,2017-05-08 00:00:00-04:00,0.027189,0.044152,NaN,NaN,NaN,NaN,NaN,NaN,0.006339
3,AAPL,2017-05-09 00:00:00-04:00,0.006339,0.033700,0.050771,NaN,NaN,NaN,NaN,NaN,-0.004676
4,AAPL,2017-05-10 00:00:00-04:00,-0.004676,0.001634,0.028867,0.045858,NaN,NaN,NaN,NaN,0.004437
...,...,...,...,...,...,...,...,...,...,...,...
95,AAPL,2017-09-19 00:00:00-04:00,0.000504,-0.007255,0.002843,-0.005638,-0.013242,-0.020731,-0.000441,0.084968,-0.016822
96,AAPL,2017-09-20 00:00:00-04:00,-0.016822,-0.016326,-0.023955,-0.014027,-0.022366,-0.036252,-0.025297,0.070302,-0.016918
97,AAPL,2017-09-21 00:00:00-04:00,-0.016918,-0.033455,-0.032968,-0.040468,-0.030707,-0.048738,-0.047734,0.066977,-0.010755
98,AAPL,2017-09-22 00:00:00-04:00,-0.010755,-0.027491,-0.043851,-0.043369,-0.050788,-0.043308,-0.022920,0.040664,-0.007907


In [ ]:
dataFinal.describe()
dataFinal.dropna(inplace = True)
#dataFinal.head()
#dataFinal.iloc[:59]
#dataFinal.drop(dataFinal.iloc[:59])

In [ ]:
dataFinal['4m_returns'].describe()

count    2235.000000
mean        0.004824
std         0.039328
min        -0.206410
25%        -0.014423
50%         0.005528
75%         0.024744
max         0.223367
Name: 4m_returns, dtype: float64

In [ ]:
dataFinal.dropna(inplace=True)
dataFinal.head()
dataFinal.isna().mean()
#dataFinal.tail()

level_0         0.0
level_1         0.0
1m_returns      0.0
2m_returns      0.0
3m_returns      0.0
4m_returns      0.0
5m_returns      0.0
10m_returns     0.0
30m_returns     0.0
60m_returns     0.0
F_1m_returns    0.0
dtype: float64

### Calculate Daily Returns

In [ ]:
# Use the `pct_change` function to calculate daily returns of `JNJ` and `MU`.
daily_returns_pctchange = df.pct_change()
daily_returns = pd.DataFrame(daily_returns_pctchange.unstack())
daily_returns.reset_index(inplace = True)
daily_returns.head()

,level_0,level_1,level_2,0
0,AAPL,close,2017-05-04 00:00:00-04:00,NaN
1,AAPL,close,2017-05-05 00:00:00-04:00,0.016514
2,AAPL,close,2017-05-08 00:00:00-04:00,0.027189
3,AAPL,close,2017-05-09 00:00:00-04:00,0.006339
4,AAPL,close,2017-05-10 00:00:00-04:00,-0.004676


In [ ]:
trading_signals_df = dataFinal.copy()
trading_signals_df['Positive Return'] = np.where(trading_signals_df['F_1m_returns'] > 0, 1.0, 0.0)

# Set x variable list of features
x_var_list = ['1m_returns','2m_returns','3m_returns','4m_returns','5m_returns','10m_returns']

# Filter by x-variable list
trading_signals_df[x_var_list] = trading_signals_df[x_var_list].shift(1)